In [1]:
%load_ext autoreload
import numpy as np
import tensorflow as tf
import sys
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import gym

In [4]:
env = gym.make("MountainCarContinuous-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [11]:
env.observation_space.shape

(2,)

In [3]:
%autoreload 2
from agents.agent import NeuralAgent

env = gym.make("MountainCarContinuous-v0")

num_episodes = 500
learning_rate = 1e-3
num_hidden = 32
memory_size = 10000
batch_size = 32
explore_mu = 0
explore_theta = 0.15
explore_sigma = 1.0

tf.reset_default_graph()
agent = NeuralAgent(state_size=env.observation_space.shape[0],
                    action_size=env.action_space.shape[0],
                    action_low=env.action_space.low[0],
                    action_high=env.action_space.high[0],
                    learning_rate=learning_rate,
                    num_hidden=num_hidden,
                    memory_size=memory_size,
                    batch_size=batch_size,
                    exploration_mu=explore_mu,
                    exploration_theta=explore_theta,
                    exploration_sigma=explore_sigma
                   )
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i_episode in range(1, num_episodes+1):
        state = env.reset()
        total_reward = 0
        while True:
            action = agent.act(sess, state)
            next_state, reward, done, _ = env.step(action)
            agent.step(sess, state, action, reward, next_state, done)
            total_reward += reward
            state = next_state
            if done:
                print("\rEpisode = {:4d}, Total Reward = {:.3f}".format(
                    i_episode, total_reward), end="")  # [debug]
                break
        sys.stdout.flush()
    saver.save(sess, "checkpoints/mountain-car.ckpt")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode =  500, Total Reward = 34.71704

In [5]:
play("MountainCarContinuous-v0", agent)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/mountain-car.ckpt
2.696767539071786


In [ ]:
def play(env_name, agent):
    env = gym.make(env_name)
    try:
        with tf.Session() as sess:
            saver.restore(sess, "checkpoints/mountain-car.ckpt")
            state = env.reset()
            env.render()
            done = False
            total_reward = 0
            while not done:
                action = agent.act(sess, state)
                next_state, reward, done, _ = env.step(action)
                total_reward += reward
                state = next_state
                env.render()
            print(total_reward)
    finally:
        env.close()

It solves mountain car!!